# 作业二：频繁模式与关联规则挖掘

**姓名**：邢晓宇  
**学号**：5720201116 
 
**GitHub**: https://github.com/OnePunchV/DataMining

# 目录

### 1. 处理数据集
  * 1.1 加载数据集  
  * 1.2 处理数据

### 2. 找出频繁项集
  * 2.1 算法实现
  * 2.2 找出频繁项集

### 3. 导出关联规则
  * 3.1 定义关联规则类
  * 3.2 算法实现
  * 3.3 找出关联规则

### 4. 评价关联规则
  * Lift指标
  * Conviction指标

In [1]:
import pandas as pd
import itertools
import collections
import numbers
import typing

# 1. 处理数据集

## 1.1 加载数据集


使用Wine Reviews数据集。

数据集主页：https://www.kaggle.com/zynicide/wine-reviews#winemag-data_first150k.csv

In [2]:
df = pd.read_csv('winemag-data_first150k.csv')
print('%d columns' % len(df.columns))
print('%d rows' % len(df.index))

11 columns
150930 rows


数据集中共有11列，150930行。数据集中的前5行展示如下。

In [3]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


## 1.2 处理数据

选取数据集中的`country`，`designation`，`province`，`variety`，`winery`这5列造transaction。将其中的缺失值用`nan`替代。为了方便查看，在每一项数据的前面加上列的名称。

处理后的transaction如下所示。共有150930条transaction。

In [4]:
columns = [
    'country',
    'designation', 
    'province', 
    'variety', 
    'winery'
]
df = df[columns].fillna('nan')

# df3 = pd.DataFrame()
# for col in df.columns:
#     df3[col] = df[col].apply(lambda x: '[' + col + '] ' + x)
transactions = df.values.tolist()

print('%d transactions.' % len(transactions), end='\n\n')
for idx, t in enumerate(transactions[:5]):
    print('%d:' % idx, t, end='\n\n')

150930 transactions.

0: ['US', "Martha's Vineyard", 'California', 'Cabernet Sauvignon', 'Heitz']

1: ['Spain', 'Carodorum Selección Especial Reserva', 'Northern Spain', 'Tinta de Toro', 'Bodega Carmen Rodríguez']

2: ['US', 'Special Selected Late Harvest', 'California', 'Sauvignon Blanc', 'Macauley']

3: ['US', 'Reserve', 'Oregon', 'Pinot Noir', 'Ponzi']

4: ['France', 'La Brûlade', 'Provence', 'Provence red blend', 'Domaine de la Bégude']



# 2. 找出频繁项集

## 2.1 算法实现

实现Apriori算法中找频繁项集部分。核心函数为`find_itemsets()`。参数`transactions`表示传入的transactions列表，`min_support`表示最小频繁项集要满足的最小支持度。

In [5]:
def join_step(itemsets: typing.List[tuple]):
    i = 0
    while i < len(itemsets):
        skip = 1
        *itemset_first, itemset_last = itemsets[i]
        tail_items = [itemset_last]
        tail_items_append = tail_items.append  # Micro-optimization
        for j in range(i + 1, len(itemsets)):
            *itemset_n_first, itemset_n_last = itemsets[j]
            if itemset_first == itemset_n_first:
                tail_items_append(itemset_n_last)
                skip += 1
            else:
                break
        itemset_first = tuple(itemset_first)
        for a, b in sorted(itertools.combinations(tail_items, 2)):
            yield itemset_first + (a,) + (b,)
        i += skip


def prune_step(
    itemsets: typing.List[tuple], 
    possible_itemsets: typing.List[tuple]
):
    itemsets = set(itemsets)
    for possible_itemset in possible_itemsets:
        for i in range(len(possible_itemset) - 2):
            removed = possible_itemset[:i] + possible_itemset[i + 1 :]
            if removed not in itemsets:
                break
        else:
            yield possible_itemset


def apriori_gen(itemsets: typing.List[tuple]):
    possible_extensions = join_step(itemsets)
    yield from prune_step(itemsets, possible_extensions)


def find_itemsets(
    transactions: typing.List[tuple],
    min_support: float,
    max_length: int = 8,
    verbosity: int = 0,
):
    transaction_sets = [set(t) for t in transactions if len(t) > 0]
    transactions = transaction_sets
    use_transaction = collections.defaultdict(lambda: True)

    if verbosity > 0:
        print("Generating itemsets.")
        print(" Counting itemsets of length 1.")

    counts = collections.defaultdict(int)
    num_transactions = 0
    for transaction in transactions:
        num_transactions += 1  # Increment counter for transactions
        for item in transaction:
            counts[item] += 1  # Increment counter for single-item itemsets

    large_itemsets = [
        (i, c)
        for (i, c) in counts.items()
        if (c / num_transactions) >= min_support
    ]

    if verbosity > 0:
        num_cand, num_itemsets = len(counts.items()), len(large_itemsets)
        print("  Found {} candidate itemsets of length 1.".format(num_cand))
        print("  Found {} large itemsets of length 1.".format(num_itemsets))

    # If large itemsets were found, convert to dictionary
    if large_itemsets:
        large_itemsets = {1: {(i,): c for (i, c) in sorted(large_itemsets)}}
    else:
        return dict(), num_transactions

    issubset = set.issubset  # Micro-optimization
    k = 2
    while large_itemsets[k - 1] and (max_length != 1):
        if verbosity > 0:
            print(" Counting itemsets of length {}.".format(k))
        itemsets_list = list(large_itemsets[k - 1].keys())
        C_k = list(apriori_gen(itemsets_list))
        C_k_sets = [set(itemset) for itemset in C_k]

        if verbosity > 0:
            print("  Found {} candidate itemsets of length {}.".format(len(C_k), k))
        if not C_k:
            break

        # Prepare counts of candidate itemsets (from the pruen step)
        candidate_itemset_counts = collections.defaultdict(int)
        if verbosity > 1:
            print("    Iterating over transactions.")
        for row, transaction in enumerate(transactions):
            if not use_transaction[row]:
                continue
            found_any = False
            for candidate, candidate_set in zip(C_k, C_k_sets):
                if issubset(candidate_set, transaction):
                    candidate_itemset_counts[candidate] += 1
                    found_any = True
            if not found_any:
                use_transaction[row] = False

        C_k = [
            (i, c)
            for (i, c) in candidate_itemset_counts.items()
            if (c / num_transactions) >= min_support
        ]
        if not C_k:
            break

        large_itemsets[k] = {i: c for (i, c) in sorted(C_k)}

        if verbosity > 0:
            num_found = len(large_itemsets[k])
            pp = "  Found {} large itemsets of length {}.".format(num_found, k)
            print(pp)
        k += 1
        if k > max_length:
            break

    if verbosity > 0:
        print("Itemset generation terminated.\n")
    return large_itemsets, num_transactions

## 2.2 找出频繁项集

找出支持度大于等于0.05的项集

In [6]:
itemsets, num_trans = find_itemsets(
    transactions=transactions,
    min_support=0.05,
    verbosity=1,
)

Generating itemsets.
 Counting itemsets of length 1.
  Found 45684 candidate itemsets of length 1.
  Found 11 large itemsets of length 1.
 Counting itemsets of length 2.
  Found 55 candidate itemsets of length 2.
  Found 8 large itemsets of length 2.
 Counting itemsets of length 3.
  Found 2 candidate itemsets of length 3.
  Found 2 large itemsets of length 3.
 Counting itemsets of length 4.
  Found 0 candidate itemsets of length 4.
Itemset generation terminated.



打印找到的频繁项集

In [8]:
for k, itemset in itemsets.items():
    print('%s itemsets:' % k)
    for items in itemset:
        print(items)
    print()

1 itemsets:
('Cabernet Sauvignon',)
('California',)
('Chardonnay',)
('France',)
('Italy',)
('Pinot Noir',)
('Red Blend',)
('Spain',)
('US',)
('Washington',)
('nan',)

2 itemsets:
('Cabernet Sauvignon', 'US')
('California', 'Pinot Noir')
('California', 'US')
('California', 'nan')
('Chardonnay', 'US')
('Pinot Noir', 'US')
('US', 'Washington')
('US', 'nan')

3 itemsets:
('California', 'Pinot Noir', 'US')
('California', 'US', 'nan')



# 3. 导出关联规则

## 3.1 定义关联规则类

首先定义关联规则类`Rule`来表示关联规则。

In [9]:
class Rule(object):
    _decimals = 3

    def __init__(
        self,
        lhs: tuple,
        rhs: tuple,
        count_full: int = 0,
        count_lhs: int = 0,
        count_rhs: int = 0,
        num_transactions: int = 0,
    ):
        self.lhs = lhs  # antecedent
        self.rhs = rhs  # consequent
        self.count_full = count_full
        self.count_lhs = count_lhs
        self.count_rhs = count_rhs
        self.num_transactions = num_transactions

    @property
    def confidence(self):
        try:
            return self.count_full / self.count_lhs
        except ZeroDivisionError:
            return None
        except AttributeError:
            return None

    @property
    def support(self):
        try:
            return self.count_full / self.num_transactions
        except ZeroDivisionError:
            return None
        except AttributeError:
            return None

    @staticmethod
    def _pf(s):
        return "{" + ", ".join(str(k) for k in s) + "}"

    def __str__(self):
        return "{} -> {}".format(self._pf(self.lhs), self._pf(self.rhs))

    def __eq__(self, other):
        return (set(self.lhs) == set(other.lhs)) and (
            set(self.rhs) == set(other.rhs)
        )

    def __hash__(self):
        return hash(frozenset(self.lhs + self.rhs))

    def __len__(self):
        return len(self.lhs + self.rhs)

## 3.2 算法实现

实现Apriori算法中找关联规则部分。核心函数为`generate_rules_apriori()`，参数`itemsets`表示找到的频繁项集，`min_confidence`表示关联规则的最小置信度。

In [10]:
def generate_rules_apriori(
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_confidence: float,
    num_transactions: int,
    verbosity: int = 0,
):
    def count(itemset):
        return itemsets[len(itemset)][itemset]

    if verbosity > 0:
        print("Generating rules from itemsets.")

    for size in itemsets.keys():
        if size < 2:
            continue
        if verbosity > 0:
            print(" Generating rules of size {}.".format(size))

        for itemset in itemsets[size].keys():
            for removed in itertools.combinations(itemset, 1):
                lhs = set(itemset).difference(set(removed))
                lhs = tuple(sorted(list(lhs)))
                conf = count(itemset) / count(lhs)
                if conf >= min_confidence:
                    yield Rule(
                        lhs,
                        removed,
                        count(itemset),
                        count(lhs),
                        count(removed),
                        num_transactions,
                    )

            H_1 = list(itertools.combinations(itemset, 1))
            yield from _ap_genrules(
                itemset, H_1, itemsets, min_confidence, num_transactions
            )

    if verbosity > 0:
        print("Rule generation terminated.\n")


def _ap_genrules(
    itemset: tuple,
    H_m: typing.List[tuple],
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_conf: float,
    num_transactions: int,
):

    def count(itemset):
        return itemsets[len(itemset)][itemset]

    if len(itemset) <= (len(H_m[0]) + 1):
        return

    H_m = list(apriori_gen(H_m))
    H_m_copy = H_m.copy()

    for h_m in H_m:
        lhs = tuple(sorted(list(set(itemset).difference(set(h_m)))))
        if (count(itemset) / count(lhs)) >= min_conf:
            yield Rule(
                lhs,
                h_m,
                count(itemset),
                count(lhs),
                count(h_m),
                num_transactions,
            )
        else:
            H_m_copy.remove(h_m)
    if H_m_copy:
        yield from _ap_genrules(
            itemset, H_m_copy, itemsets, min_conf, num_transactions
        )

## 3.3 找出关联规则

找出关联规则，设置最小支持度为0.1。

In [11]:
rules = generate_rules_apriori(
    itemsets=itemsets, 
    min_confidence=0.1, 
    num_transactions=num_trans, 
    verbosity=1,
)
rules = list(rules)

Generating rules from itemsets.
 Generating rules of size 2.
 Generating rules of size 3.
Rule generation terminated.



展示找到的关联规则，以及它们的支持度和置信度。

In [12]:
rules_t = []
for rule in rules:
    valid = True
    for val in rule.lhs + rule.rhs:
        if 'nan' in val:
            valid = False
            break
    if valid:
        rules_t.append(rule)
rules = rules_t

rules_df = pd.DataFrame({
    'Rules': rules,
    'Left': list(map(lambda x: x.lhs, rules)),    
    'Right': list(map(lambda x: x.rhs, rules)),
    'Support': list(map(lambda x: x.support, rules)),
    'Confidence': list(map(lambda x: x.confidence, rules)),
})
rules_df

,Rules,Left,Right,Support,Confidence
0,{US} -> {Cabernet Sauvignon},"(US,)","(Cabernet Sauvignon,)",0.060856,0.147203
1,{Cabernet Sauvignon} -> {US},"(Cabernet Sauvignon,)","(US,)",0.060856,0.716850
2,{Pinot Noir} -> {California},"(Pinot Noir,)","(California,)",0.050633,0.534069
3,{California} -> {Pinot Noir},"(California,)","(Pinot Noir,)",0.050633,0.171699
4,{US} -> {California},"(US,)","(California,)",0.294892,0.713304
5,{California} -> {US},"(California,)","(US,)",0.294892,1.000000
6,{US} -> {Chardonnay},"(US,)","(Chardonnay,)",0.053866,0.130295
7,{Chardonnay} -> {US},"(Chardonnay,)","(US,)",0.053866,0.561270
8,{US} -> {Pinot Noir},"(US,)","(Pinot Noir,)",0.068601,0.165937
9,{Pinot Noir} -> {US},"(Pinot Noir,)","(US,)",0.068601,0.723601


# 4. 评价关联规则

## 4.1 Lift指标

In [13]:
def lift(rule):
    observed_support = rule.count_full / rule.num_transactions
    prod_counts = rule.count_lhs * rule.count_rhs
    expected_support = (prod_counts) / rule.num_transactions ** 2
    return observed_support / expected_support

In [14]:
rules_df['Lift'] = list(map(lambda x: lift(x), rules))
rules_df

,Rules,Left,Right,Support,Confidence,Lift
0,{US} -> {Cabernet Sauvignon},"(US,)","(Cabernet Sauvignon,)",0.060856,0.147203,1.733964
1,{Cabernet Sauvignon} -> {US},"(Cabernet Sauvignon,)","(US,)",0.060856,0.716850,1.733964
2,{Pinot Noir} -> {California},"(Pinot Noir,)","(California,)",0.050633,0.534069,1.811070
3,{California} -> {Pinot Noir},"(California,)","(Pinot Noir,)",0.050633,0.171699,1.811070
4,{US} -> {California},"(US,)","(California,)",0.294892,0.713304,2.418866
5,{California} -> {US},"(California,)","(US,)",0.294892,1.000000,2.418866
6,{US} -> {Chardonnay},"(US,)","(Chardonnay,)",0.053866,0.130295,1.357638
7,{Chardonnay} -> {US},"(Chardonnay,)","(US,)",0.053866,0.561270,1.357638
8,{US} -> {Pinot Noir},"(US,)","(Pinot Noir,)",0.068601,0.165937,1.750293
9,{Pinot Noir} -> {US},"(Pinot Noir,)","(US,)",0.068601,0.723601,1.750293


## 4.2 Conviction指标

In [15]:
def conviction(self):
    eps = 10e-10  # Avoid zero division
    prob_not_rhs = 1 - self.count_rhs / self.num_transactions
    prob_not_rhs_given_lhs = 1 - self.confidence
    return prob_not_rhs / (prob_not_rhs_given_lhs + eps)

In [16]:
rules_df['Conviction'] = list(map(lambda x: conviction(x), rules))
rules_df

,Rules,Left,Right,Support,Confidence,Lift,Conviction
0,{US} -> {Cabernet Sauvignon},"(US,)","(Cabernet Sauvignon,)",0.060856,0.147203,1.733964,1.073064e+00
1,{Cabernet Sauvignon} -> {US},"(Cabernet Sauvignon,)","(US,)",0.060856,0.716850,1.733964,2.071635e+00
2,{Pinot Noir} -> {California},"(Pinot Noir,)","(California,)",0.050633,0.534069,1.811070,1.513334e+00
3,{California} -> {Pinot Noir},"(California,)","(Pinot Noir,)",0.050633,0.171699,1.811070,1.092833e+00
4,{US} -> {California},"(US,)","(California,)",0.294892,0.713304,2.418866,2.459424e+00
5,{California} -> {US},"(California,)","(US,)",0.294892,1.000000,2.418866,5.865832e+08
6,{US} -> {Chardonnay},"(US,)","(Chardonnay,)",0.053866,0.130295,1.357638,1.039465e+00
7,{Chardonnay} -> {US},"(Chardonnay,)","(US,)",0.053866,0.561270,1.357638,1.337004e+00
8,{US} -> {Pinot Noir},"(US,)","(Pinot Noir,)",0.068601,0.165937,1.750293,1.085284e+00
9,{Pinot Noir} -> {US},"(Pinot Noir,)","(US,)",0.068601,0.723601,1.750293,2.122230e+00
